## **Library**

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
import torch
from torch import Tensor, device

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
corpus = pd.read_csv("/content/preprocessed_data")
corpus.drop(corpus.filter(regex="Unnamed"),axis=1, inplace=True)

In [3]:
corpus = corpus[:1000]

In [4]:
corpus['Text'] = corpus['Text'].replace(np.nan, '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Tokens

In [5]:
def create_tokens(data):   
    sent_tokens =[]
    word_tokens = []
    for i in range(len(data)):
        word_tokens.append(word_tokenize(data[i]))
        sent_tokens.append(word_tokens[i])
    return sent_tokens

In [6]:
docs=corpus["Text"].tolist()
sentences=create_tokens(docs)
len(sentences)

1000

In [7]:
k = []
for i in range(1000):
  for j in range(len(sentences[i])):
    k.append(sentences[i][j])
words = set(k)     
len(words)

5278

In [8]:
word2int = {}
int2word = {}
vocab_size = len(words)

for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [ ]:
vectorizer = CountVectorizer(min_df=0, token_pattern=r"\b\w+\b")
vectorizer.fit(docs)
print(vectorizer.vocabulary_)
vector = vectorizer.transform(docs)

{'moeller': 3032, 'student': 4592, 'run': 4121, 'newspaper': 3174, 'the': 4765, 'crusader': 1131, 'consistently': 1003, 'recognized': 3883, 'one': 3287, 'top': 4843, 'region': 3925, 'in': 2301, 'first': 1837, 'place': 3530, 'second': 4223, 'highest': 2173, 'honor': 2203, 'high': 2171, 'school': 4194, 'receive': 3876, 'american': 191, 'scholastic': 4193, 'press': 3642, 'association': 327, 'ref': 3904, 'squire': 4501, 'literary': 2734, 'journal': 2516, 'features': 1783, 'stories': 4568, 'poems': 3564, 'essays': 1614, 'written': 5251, 'students': 4593, 'paul': 3446, 'keels': 2554, 'play': 3548, 'announcer': 227, 'ohio': 3276, 'state': 4527, 'university': 4995, 'football': 1872, 'basketball': 430, 'wbns': 5166, 'fm': 1859, 'joe': 2507, 'uecker': 4953, 'senator': 4263, 'r': 3805, 'rico': 4063, 'murray': 3092, 'undrafted': 4970, 'free': 1920, 'agent': 118, 'signee': 4362, 'cincinnati': 793, 'bengals': 476, 'tony': 4839, 'hunter': 2230, 'professional': 3707, 'player': 3550, 'buffalo': 599, 'b

In [ ]:
print(vectorizer.vocabulary_)
print(vector.shape)

{'moeller': 3032, 'student': 4592, 'run': 4121, 'newspaper': 3174, 'the': 4765, 'crusader': 1131, 'consistently': 1003, 'recognized': 3883, 'one': 3287, 'top': 4843, 'region': 3925, 'in': 2301, 'first': 1837, 'place': 3530, 'second': 4223, 'highest': 2173, 'honor': 2203, 'high': 2171, 'school': 4194, 'receive': 3876, 'american': 191, 'scholastic': 4193, 'press': 3642, 'association': 327, 'ref': 3904, 'squire': 4501, 'literary': 2734, 'journal': 2516, 'features': 1783, 'stories': 4568, 'poems': 3564, 'essays': 1614, 'written': 5251, 'students': 4593, 'paul': 3446, 'keels': 2554, 'play': 3548, 'announcer': 227, 'ohio': 3276, 'state': 4527, 'university': 4995, 'football': 1872, 'basketball': 430, 'wbns': 5166, 'fm': 1859, 'joe': 2507, 'uecker': 4953, 'senator': 4263, 'r': 3805, 'rico': 4063, 'murray': 3092, 'undrafted': 4970, 'free': 1920, 'agent': 118, 'signee': 4362, 'cincinnati': 793, 'bengals': 476, 'tony': 4839, 'hunter': 2230, 'professional': 3707, 'player': 3550, 'buffalo': 599, 'b

### Creating training data(one hot encoder)

In [ ]:
data = []
WINDOW_SIZE = 3
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

In [ ]:
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

In [ ]:
x_train = [] 
for data_word in data:
    x_train.append(to_one_hot(word2int[data_word[0]], vocab_size))

In [ ]:
y_train = [] 
for data_word in data:
    y_train.append(to_one_hot(word2int[data_word[1]], vocab_size))

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

### Embedding Size 

In [ ]:
EMBEDDING_DIM = 20

In [ ]:
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM]))
hidden_representation = tf.add(tf.matmul(x,W1), b1)

In [ ]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

### Training model of word2vec

In [ ]:
#sess = tf.Session()
#init = tf.global_variables_initializer()
#sess.run(init) 
#cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
#train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
#n_iters = 1000
#for _ in range(n_iters):
#    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
#    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

In [ ]:
#vectors = sess.run(W1 + b1)

### Loading the training vectors(embedding)

In [9]:
import pickle

In [10]:
with open('/content/vectors.pickle1', 'rb') as handle:
    vectors = pickle.load(handle)

### Average of vectors

In [11]:
avg_data = []
for row in docs:
    vec = np.zeros(20)
    count = 0
    for word in row:
        try:
            vec += vectors[word2int[word]]
            count += 1
        except:
            pass
    avg_data.append(vec/count)

In [12]:
avg_vec_wv = np.vstack(avg_data)

### TF-IDF

In [13]:
word_set = list(words)
total_documents = len(docs)
total_documents

1000

In [14]:
index_dict = {} #Dictionary to store index for each word
i = 0
for word in word_set:
    index_dict[word] = i
    i += 1

In [15]:
#Create a count dictionary
def count_dict(sentences):
    word_count = {}
    for word in word_set:
        word_count[word] = 0
        for sent in sentences:
            if word in sent:
                word_count[word] += 1
    return word_count

In [16]:
word_count = count_dict(docs)

In [17]:
#Term Frequency
def termfreq(document, word):
    N = len(document)
    occurance = len([token for token in document if token == word])
    return occurance/N

In [18]:
#Inverse Document Frequency
def inverse_doc_freq(word):
    try:
        word_occurance = word_count[word] + 1
    except:
        word_occurance = 1
    return np.log(total_documents/word_occurance)

In [19]:
#TF-IDF
def tf_idf(sentence):                
    tf_idf_vec = np.zeros((len(word_set),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)         
        value = tf*idf
        tf_idf_vec[index_dict[word]] = value 
    return tf_idf_vec

In [20]:
tf_idf_vector = []
for sent in sentences:
    vec = tf_idf(sent)
    tf_idf_vector.append(vec)

In [21]:
tf_idf_vectors = np.vstack(tf_idf_vector)

### TF-IDF WORD2VEC

In [22]:
tf_w_data = []
i = 0
for row in sentences:
    vec = [0 for i in range(20)]    
    temp_tfidf = []
    for val in tf_idf_vectors[i]:
        if val != 0:
            temp_tfidf.append(val)    
    count = 0
    tf_idf_sum = 0
    for word in row:
        try:
            count += 1
            tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
            vec += (temp_tfidf[count-1] * vectors[word2int[word]])
        except:
          pass
    vec = (float)(1/tf_idf_sum) * vec
    tf_w_data.append(vec)
    i = i + 1 

In [23]:
final_vectors = np.vstack(tf_w_data)

### Cosine Similarity

In [24]:
def cos_sim(a: Tensor, b: Tensor):
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)
    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

### Creating the clusters

In [25]:
def cluster_detection(embeddings, threshold, min_community_size, batch_size=32):
    if not isinstance(embeddings, torch.Tensor):
        embeddings = torch.tensor(embeddings)
    threshold = torch.tensor(threshold, device=embeddings.device)
    extracted_communities = []
    min_community_size = min(min_community_size, len(embeddings))
    sort_max_size = min(max(2 * min_community_size, 50), len(embeddings))
    for start_idx in range(0, len(embeddings), batch_size):
        cos_scores = cos_sim(embeddings[start_idx:start_idx + batch_size], embeddings)
        top_k_values, _ = cos_scores.topk(k=min_community_size, largest=True)
        for i in range(len(top_k_values)):
            if top_k_values[i][-1] >= threshold:
                new_cluster = []
                top_val_large, top_idx_large = cos_scores[i].topk(k=sort_max_size, largest=True)
                while top_val_large[-1] > threshold and sort_max_size < len(embeddings):
                    sort_max_size = min(2 * sort_max_size, len(embeddings))
                    top_val_large, top_idx_large = cos_scores[i].topk(k=sort_max_size, largest=True)
                for idx, val in zip(top_idx_large.tolist(), top_val_large):
                    if val < threshold:
                        break
                    new_cluster.append(idx)
                extracted_communities.append(new_cluster)
        del cos_scores
    extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)
    unique_communities = []
    extracted_ids = set()
    for cluster_id, community in enumerate(extracted_communities):
        community = sorted(community)
        non_overlapped_community = []
        for idx in community:
            if idx not in extracted_ids:
                non_overlapped_community.append(idx)
        if len(non_overlapped_community) >= min_community_size:
            unique_communities.append(non_overlapped_community)
            extracted_ids.update(non_overlapped_community)
    unique_communities = sorted(unique_communities, key=lambda x: len(x), reverse=True)
    return unique_communities

In [43]:
clusters = cluster_detection(final_vectors, min_community_size=1, threshold=0.50)

In [44]:
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", docs[sentence_id])


Cluster 1, #103 Elements 
	 the effects disequilibrium vary according equations belong
	 the equation economic equilibrium disequilibrium s t m i g x five sector model levels income expenditure output greatly rise causing boom economic activity
	 in real flow household sector supplies raw material land labour capital enterprise firms return firms sector provides finished goods services household sector

Cluster 2, #57 Elements 
	 the first visualize modern circular flow income model frank knight publication the economic organization ref
	 laser doppler vibrometry a laser doppler vibrometer used produce accurate deflection measurements oscillating cantilever ref thus used non contact mode
	 due nature afm probes cannot normally measure steep walls overhangs

Cluster 3, #51 Elements 
	 tony hunter professional football player buffalo bills los angeles rams
	 the lacrosse team two state titles constant force state midwest
	 marx distinguishes simple reproduction expanded enlarged reprodu

In [45]:
cluster_list = []
for i, cluster in enumerate(clusters):
  cluster_list.append(cluster)

In [46]:
sen = []
for i, cluster in enumerate(clusters):
  for id in cluster[:]:
    sen.append([i, docs[id]])

### Saving in csv format

In [47]:
df=pd.DataFrame(sen,columns=['cluster_no','sentences'])

In [48]:
df.head(20)

,cluster_no,sentences
0,0,the effects disequilibrium vary according equa...
1,0,the equation economic equilibrium disequilibri...
2,0,in real flow household sector supplies raw mat...
3,0,the watts family also strongly connected beale...
4,0,in november gavin sullivan paul nicholas revea...
5,0,the common method cantilever deflection measur...
6,0,stm the first atomic microscope used stm compl...
7,0,optical interferometry optical interferometry ...
8,0,operation mode image forming afm generally cla...
9,0,just like tool afm usefulness limitations


In [50]:
df.to_csv('cluster_using_word2vec_tfidf.csv')